In [ ]:
from chart_ipynb import utils
from chart_ipynb.chart_framework import ChartSuperClass
import numpy as np
import pandas as pd
import pandas_datareader
import pandas_datareader.data as web
import datetime
import time

In [ ]:
api_key = '1JFowowyzc-FnajAsDkY'
start = datetime.datetime(2017,1,1)
end = datetime.datetime(2018,1,1)
aapl = web.DataReader('AAPL',"quandl", start, end, api_key = api_key)
amzn = web.DataReader('AMZN',"quandl", start, end, api_key = api_key)
googl = web.DataReader('GOOGL',"quandl", start, end, api_key = api_key)

In [ ]:
aapl.head()

In [ ]:
amzn.head()

In [ ]:
googl.head()

In [ ]:
col = 'Close'
aapl_col = aapl[col]
amzn_col = amzn[col]
googl_col = googl[col]

In [ ]:
def data_format(dataset, val_col):
    """
    dataset: pd.DataFrame
    val_col: the column name for the target value. e.g 'Close'
    """
    data = dataset[val_col]
    idx_reset_df = dataset.reset_index()
    if 'Date' not in idx_reset_df.columns:
        return 'please rename the date columns to "Date"'
    sort_df = idx_reset_df.sort_values(by='Date')
    sort_df['Date']=sort_df['Date'].astype(str)
    return  list(sort_df[val_col]), list(sort_df['Date'])

In [ ]:
# df1 = aapl_col.reset_index().rename(columns={'Date':'t',"Close": "y"})
# df1 = df1.sort_values(by='t')
# df1['t']=df1['t'].apply(lambda x: int(time.mktime(x.timetuple())*1000))
# df1['t']=df1['t'].astype(str)
# data1 = df1.to_dict('r')
# val1 = list(df1['y'])
# label1 = list(df1['t'])

In [ ]:
aapl_val, aapl_label = data_format(aapl, 'Close')
amzn_val, amzn_label = data_format(amzn, 'Close')
googl_val, googl_label = data_format(googl, 'Close')

In [ ]:
dataset1 = utils.dataset(
                label = 'AAPL',
                backgroundColor = utils.color_rgb('red',0.5),
                borderColor = utils.color_rgb('red'),
                data = aapl_val,
                type = 'line',
                pointRadius = 0,
                fill = False,
                lineTension = 0,
                borderWidth = 2
            )
dataset2 = utils.dataset(
                label = 'AMZN',
                backgroundColor = utils.color_rgb('blue',0.5),
                borderColor = utils.color_rgb('blue'),
                data = amzn_val,
                type = 'line',
                pointRadius = 0,
                fill = False,
                lineTension = 0,
                borderWidth = 2
            )
dataset3 = utils.dataset(
                label = 'GOOGL',
                backgroundColor = utils.color_rgb('green',0.5),
                borderColor = utils.color_rgb('green'),
                data = googl_val,
                type = 'line',
                pointRadius = 0,
                fill = False,
                lineTension = 0,
                borderWidth = 2
            )
data = utils.data(
    labels = aapl_label,
    datasets = [dataset1,dataset2,dataset3]
)

In [ ]:
config = utils.config(
        type = 'line',
        data = data,
        options = utils.options(
            animation = {
                'duration': 0
            },
            scales = {
                'xAxes': [{
                    'display':True,
                    'scaleLabel':{
                        'display':True,
                        'labelString':'Date'
                    }
#                     'type': 'time',
#                     'distribution': 'series',
#                     'offset': True,
                    ,'ticks': {
                        'major': {
                            'enabled': True,
                            'fontStyle': 'bold'
                        },
                        'source': 'data',
                        'autoSkip': True,
                        'autoSkipPadding': 10,
                        'maxRotation': 60,
#                         'sampleSize': 100
                    },
#                     afterBuildTicks: function(scale, ticks) {
#                         var majorUnit = scale._majorUnit;
#                         var firstTick = ticks[0];
#                         var i, ilen, val, tick, currMajor, lastMajor;

#                         val = moment(ticks[0].value);
#                         if ((majorUnit === 'minute' && val.second() === 0)
#                                 || (majorUnit === 'hour' && val.minute() === 0)
#                                 || (majorUnit === 'day' && val.hour() === 9)
#                                 || (majorUnit === 'month' && val.date() <= 3 && val.isoWeekday() === 1)
#                                 || (majorUnit === 'year' && val.month() === 0)) {
#                             firstTick.major = true;
#                         } else {
#                             firstTick.major = false;
#                         }
#                         lastMajor = val.get(majorUnit);

#                         for (i = 1, ilen = ticks.length; i < ilen; i++) {
#                             tick = ticks[i];
#                             val = moment(tick.value);
#                             currMajor = val.get(majorUnit);
#                             tick.major = currMajor !== lastMajor;
#                             lastMajor = currMajor;
#                         }
#                         return ticks;
#                     }
                }],
                'yAxes': [{
                    'gridLines': {
                        'drawBorder': False
                    },
                    'scaleLabel': {
                        'display': True,
                        'labelString': 'Closing price ($)'
                    }
                }]
            },
        )
)


In [ ]:
line_chart = ChartSuperClass()
line_chart.initialize_chart(width=800, config=config)
line_chart

In [1]:
from chart_ipynb import line

In [2]:
start = '2017-1-1'
end = '2018-1-1'
symbols = ['AAPL','AMZN','GOOGL']
col = 'Open'
line.time_series_stock(symbols, start, end, col)

Line(status='deferring flush until render')